# 1. Data Review & Cleaning


# Import libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
import datetime
import math as m
import scipy.stats as stats
import warnings 
import statsmodels.api as sm

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None) 


from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error,mean_absolute_error

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn import linear_model
from sklearn.metrics import accuracy_score, precision_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor
from math import sqrt

KeyboardInterrupt: 

In [ ]:
df = pd.read_csv("Perth.csv")
#Quickly look
df.head()

In [ ]:
from pandas_profiling import ProfileReport
import pandas_profiling as pdp
profile = ProfileReport(df, title="Price house", 
                       minimal=True, progress_bar=False,
                       missing_diagrams ={
                           "heatmap": False,
                           "dendrogram": False
                       })

In [ ]:
profile

In [ ]:
#I changed capitalization to lowercase and replace two columns
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace("nearest_sch_dist", "distance_nearest_school")
df.columns = df.columns.str.replace("nearest_stn_dist", "distance_nearest_station")
df.columns = df.columns.str.replace("cbd_dist","distance_to_city_center")

In [ ]:
#There is multicollinearity in this 3 variables so I dropped

df.drop(["nearest_sch_rank", "nearest_sch","nearest_stn"], axis=1, inplace=True)

In [ ]:
#Checked for duplicated values
df.duplicated().sum()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
#Checked the nans in all the dataset
df.isna().sum()/len(df)

### After I'm going to replace the NaN values of  build_year because represents almost 10% of my dataset and garage 7.3%.

# Price will be my target value because I want to predict the housing price


In [ ]:
df["price"]

### In which suburb have the most houses been sold?

In [ ]:
df["suburb"].value_counts()

In [ ]:
#there are clearly evidence that from the last 20 years we have a increase of 850%(7 houses ---->1990 to 5261---> 2020)
df["date_sold"].value_counts()

In [ ]:
#I changed the datetype to only the year

df["date_sold"] = pd.to_datetime(df["date_sold"], errors = "coerce")
df["date_sold"] = df["date_sold"].apply(lambda x: x.strftime("%Y"))

df["date_sold"]

In [ ]:
#And then changed the type to numerical
df["date_sold"] =df["date_sold"].astype("int64")
df["date_sold"]

### Log-transformation on *build_year*

In [ ]:
### Log Transformation on build_year to see if the NaN changes the format to gaussian distribution because now are skew
log_build_year=np.log(df["build_year"])

fig, ax = plt.subplots(1, 2, figsize=(10,5), dpi=100)
sns.histplot(x=df["build_year"], kde=True, ax=ax[0])
sns.histplot(x=log_build_year, kde=True, ax=ax[1])
ax[0].set_title("Before Log-transformation")
ax[1].set_title("After Log-transformation")
plt.show()

### After the logarithmical transformation the plot doesn't look like Gaussian distribution....
### I checked the mean of the logarithmical and is too low (7.6 underfits a lot... when the normal mean is 2003
# So I decided to replace the NaN values with the mean of build_year

In [ ]:
df["build_year"].fillna((df["build_year"].mean()), inplace = True)
df["build_year"].isna().sum()

### Log-transformation on  *Garage*

In [ ]:
#Checked Log Transformation on garage
log_garage=np.log(df["garage"])

fig, ax = plt.subplots(1, 2, figsize=(10,5), dpi=100)
sns.histplot(x=df["garage"], kde=True, ax=ax[0])
sns.histplot(x=log_garage, kde=True, ax=ax[1])
ax[0].set_title('Before Log-transformation')
ax[1].set_title('After Log-transformation')
plt.show()

### Log-transformation looks gaussian, So I decided to replace the NaN values with the mean of log_garage


In [ ]:
df["garage"].fillna((log_garage.mean()), inplace = True)

In [ ]:
#Checked the NaNs
df.isna().sum()/len(df)

In [ ]:
#I changed it to numerical because if not give me error when I try to substract
df["build_year"].astype("int64").round(2)

In [ ]:
#Feature engineering
df["house_age"]= df["date_sold"] - df["build_year"]

In [ ]:
#Checked numerical values
numerical =df.select_dtypes(include=np.number)
numerical

In [ ]:
df["date_sold"].value_counts()

In [ ]:
#Univariate analysis of continuos varaibles
for variables in numerical:
    plt.figure(figsize=(16,6))
    sns.distplot(df, x=df[variables], kde=True)
    plt.show()

In [ ]:
#Checked categorical values
categorical = df.select_dtypes(object)
categorical

# 2. Exploratory Data Analysis


In [ ]:
correlation = df.corr()
plt.figure(figsize=(22,18))
sns_plot =sns.heatmap(correlation, cmap="YlGnBu",annot = True)  
figure = sns_plot.get_figure()
plt.show()

### Visualizing the location of the houses based on latitude and longitude.


In [ ]:
plt.figure(figsize=(10,10))
sns.jointplot(data=df, x=df["latitude"], y=df["longitude"])
plt.ylabel("latitude", fontsize=12) 
plt.xlabel("longitude", fontsize=12) 
plt.show()


### The sold houses are higher in longitude between 115-116 and latitude between 31.8-32.4



In [ ]:
px.histogram(df, x="price", title="Distribution of House price")

In [ ]:
px.box(df, x="price", title="Boxplot of House price")

In [ ]:
px.density_contour(df, x="price", y ="distance_to_city_center", marginal_x="histogram")

In [ ]:
fig5 = px.scatter(df, x=df["bedrooms"], y=df["price"],title="Price Room")
fig5.show()

### With this graph we want to know which is the most common house (Bedroom wise)


In [ ]:
df["bedrooms"].value_counts().plot(kind="bar")
plt.title("number of Bedroom")
plt.xlabel("bedrooms")
plt.ylabel("Count")
sns.despine


In [ ]:
px.histogram(df, x="bedrooms", title="the most common house Bedroom wise")

### The highest seller's are  4 bedroom's 

In [ ]:
px.density_contour(df, x="price", y ="bedrooms",title="Price vs Bedrooms", marginal_x="histogram")

### there are some irregularaties f.e 10 bedrooms 405k and 2 million per only one bed (a part of this the plot looks trustworthy)

In [ ]:
fig3 = px.scatter(df, x=df["bedrooms"], y=df["distance_to_city_center"],title="Is the price higher if the city center are near?")
fig3.show()

In [ ]:
fig3333 = px.scatter(df, x=df["bedrooms"], y=df["distance_nearest_school"],title="Rooms near school")
fig3333.show()

In [ ]:
fig234 = px.scatter(df, x=df["house_age"], y=df["price"],title="Years")
fig234.show()

In [ ]:
fig2 = px.scatter(df, x=df["price"], y=df["distance_nearest_station"],title="Is the price cheaper near the station?")
fig2.show()

In [ ]:
fig4 = px.scatter(df, x=df["price"], y=df["land_area"],title="How much cost the land area?")
fig4.show()

In [ ]:
fig456 = px.scatter(df, x=df["price"], y=df["floor_area"],title="price vs floor area")
fig456.show()

In [ ]:
px.density_contour(df, x="price", y ="floor_area", marginal_x="histogram")

In [ ]:
px.density_contour(df, x="price", y ="suburb", marginal_x="histogram")

### With this graph we want to check where are the best profits

In [ ]:
px.histogram(df, x="suburb", title="Which suburb are more sales?") 

### Which post code has more transactions?

In [ ]:
px.histogram(df, x="postcode", title="Which post code has more transactions...")

In [ ]:
px.histogram(df, x="build_year", title="Which year they construct more houses?")

In [ ]:
px.histogram(df, x="house_age", title="Age of the house when they sold it")

In [ ]:
px.histogram(df, x="date_sold", title="Which year sold more houses?")

In [ ]:
px.histogram(df, x="garage", title="Number of garages...")

### Geomap

In [ ]:
from arcgis.gis import GIS
from arcgis.geocoding import geocode
from arcgis.features import GeoAccessor, GeoSeriesAccessor
gis = GIS()

In [ ]:
#I got the 1% of my dataset random
df2=df.sample(frac=0.01)

In [ ]:
#preparing the data
df3= pd.DataFrame.spatial.from_xy(df2, "longitude","latitude")

In [ ]:
#Map of the properties
property_map = gis.map("Melbourne, perth")
property_map.basemap = "streets"
property_map 
#
df3.spatial.plot(map_widget=property_map)

In [ ]:
#See 1% of the sold houses randomly
property_map 

# Preprocessing

In [ ]:
#I dropped price because is my target value.
# Address and Suburb because are categorical and discrete values
X= df.drop(["price","address","suburb"], axis=1) 
y=df["price"]

In [ ]:
#checked the columns
X.columns

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.32,random_state=41)

In [ ]:
#checking number of rows
print("Nb of rows of X_train = {}".format(len(X_train)))
print("Nb of rows of X_test = {}".format(len(X_test)))
print("Nb of rows of y_train = {}".format(len(y_train)))
print("Nb of rows of y_test = {}".format(len(y_test)))

In [ ]:
reg = LinearRegression()
scaler1 = StandardScaler()
scaler2 = MinMaxScaler()
scaler3 = PolynomialFeatures(degree=2, interaction_only=True)
scaler4 = RobustScaler()
ridge = Ridge(alpha=0.3)
lasso = Lasso(alpha=0.1)

### Baseline Model

In [ ]:
LinReg = reg.fit(X_train, y_train) #training the model

# predicting y with X_test
y_pred=LinReg.predict(X_test) 

r2 = r2_score(y_test, y_pred) # calculating r2 
mean_sq_err = mean_squared_error(y_test, y_pred) # calculating mean squared error
mean_abs_err = mean_absolute_error(y_test, y_pred) # calculating absolute error

print("The metrics of the basic model are the following:")
print("R2 score: ", round(r2,4))
print("MAE: ", round(mean_abs_err,4))
print("MSE: ", round(mean_sq_err,4))
print("RMSE: ", round(np.sqrt(mean_sq_err),4))

In [ ]:
#Preparing the model
def model_inplace(scaler, model, X_train, X_test, y_train, y_test):
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    model.fit(X_train,y_train)
    r2 = r2_score(y_test, y_pred)
    mean_sq_err = mean_squared_error(y_test, y_pred) 
    mean_abs_err = mean_absolute_error(y_test, y_pred)
    print("R2 score on train set: ", round(model.score(X_train, y_train),4))
    print("R2 score on test set: ", round(model.score(X_test, y_test),4))
    #print("MAE: ", round(mean_abs_err,4))
    #print("MSE: ", round(mean_sq_err,4))
    #print("RMSE: ", round(np.sqrt(mean_sq_err),4))

### StandardScaler

In [ ]:
model_inplace(scaler1, reg, X_train, X_test, y_train, y_test)

### MinMaxScaler

In [ ]:
model_inplace(scaler2, reg, X_train, X_test, y_train, y_test)

### PolynomialFeatures

In [ ]:
model_inplace(scaler3, reg, X_train, X_test, y_train, y_test)

### RobustScaler

In [ ]:
model_inplace(scaler4, reg, X_train, X_test, y_train, y_test)

In [ ]:
#the best model using the scaler is PolynomialFeatures

### Ridge

In [ ]:
model_inplace(scaler1, ridge, X_train, X_test, y_train, y_test)#Ridge with Standardscaler

In [ ]:
model_inplace(scaler2, ridge, X_train, X_test, y_train, y_test)#Ridge with Minmax

In [ ]:
model_inplace(scaler3, ridge, X_train, X_test, y_train, y_test)#Ridge with Polynomial

In [ ]:
model_inplace(scaler4, ridge, X_train, X_test, y_train, y_test)#Ridge with Robustscaler

In [ ]:
#Ridge with polynomial feature is better than the other scalers

### Lasso

In [ ]:
model_inplace(scaler1, lasso, X_train, X_test, y_train, y_test)#Lasso with Standardscaler

In [ ]:
model_inplace(scaler2, lasso, X_train, X_test, y_train, y_test)#Lasso with Minmax

In [ ]:
model_inplace(scaler3, lasso, X_train, X_test, y_train, y_test)#Lasso with Polynomial

In [ ]:
model_inplace(scaler4, lasso, X_train, X_test, y_train, y_test)#Lasso with Robustscaler

In [ ]:
#Lasso with Polynomial feature is better than the other scalers

### Random forest

In [ ]:
n_estimators = [10,100,1000]

# Model with a random forest 
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=100)
rf.fit(X_train, y_train)

In [ ]:
print("R2 score on train set:", rf.score(X_train, y_train))
print("R2 score on test set:", rf.score(X_test, y_test))

### Hyperparametertunning using Random Forest

In [ ]:
rfg = RandomForestRegressor() # default is alpha=1.0
params = {"n_estimators":  np.arange(10,100,30)}
grid = GridSearchCV(rfg,  param_grid=params, cv=10, verbose=1)

grid.fit(X_train, y_train)
best_model = grid.best_estimator_
#print("grid.best_estimator:", params.score(X_train, y_train))
print("R2 score on train set:", best_model.score(X_train, y_train))
print("R2 score on test set:", best_model.score(X_test, y_test))



# np.arange(10,100,100): 10--->40--->70--->100   using 10 Folds-CV

### GradientBoostingRegressor


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor(random_state=0).fit(X_train,y_train)

In [ ]:
print("R2 score on train set:", gbr.score(X_train, y_train))
print("R2 score on test set:", gbr.score(X_test, y_test))

### Comparing the best R2 score

In [ ]:
print("R2 score on my baseline model : ", round(r2,4))
print("R2 score on Randomforest train set:", rf.score(X_train, y_train))
print("R2 score on Randomforest test set:", rf.score(X_test, y_test))
print("R2 score with Hyperparametertunning train set:", best_model.score(X_train, y_train))
print("R2 score with Hyperparametertunning test set:", best_model.score(X_test, y_test))
print("R2 score on GradientBoostingRegressor train set:", gbr.score(X_train, y_train))
print("R2 score on GradientBoostingRegressor test set:", gbr.score(X_test, y_test))

In [ ]:
#Baseline Model and the best 3 models
Score = pd.DataFrame({"Metric":["R2 score Baselinemodel","R2 score on RandomForestRegressor","R2 score on Hyperparametertunning","R2 score on Gradientboosting"],
           "Train":[0.5771,rf.score(X_train, y_train),best_model.score(X_train, y_train),gbr.score(X_train, y_train)],
           "Test":[0,rf.score(X_test, y_test),best_model.score(X_test, y_test),gbr.score(X_test, y_test)]})
                   
                   
Score               
                   

### Feature importance using Random Forest Regressor

In [ ]:
X_features = X.columns.to_list()

In [ ]:
X_features

In [ ]:
rf.feature_importances_

In [ ]:
feature_ranking = pd.DataFrame({
    "features":X_features, "coefficients": rf.feature_importances_
})

In [ ]:
feature_ranking = rf.feature_importances_.argsort()

In [ ]:
fig = px.bar(feature_ranking, y = X_features, x = rf.feature_importances_, orientation='h', title = "Feature Importance using Random Forest Regressor ",
            labels = {"x":"Coefficients","y":"Features" })
fig.update_layout(barmode="stack",yaxis={"categoryorder":"total ascending"})
fig.show()

### Prediction using Random Forest Regressor

In [ ]:
#I created the predictions with the model Random Forest Regressor
y_pred_train = rf.predict(X_train)
y_pred_test = rf.predict(X_test)

In [ ]:
#Checked the size of X_train
y_pred_train.size

In [ ]:
#Checked the size of X_test
y_pred_test.size

In [ ]:
#I created a plot for the Groundtruth and Predictions test

#X_train
fig, ax = plt.subplots(2,1)
sns.scatterplot(x=y_train,y=y_pred_train,ax=ax[0]);
ax[0].plot(y_train,y_train, color= "black")
ax[0].set_xlabel("Real Price")
ax[0].set_ylabel("Predicted Price")
ax[0].set_title("Train Set")

#X_test
sns.scatterplot(x=y_test,y=y_pred_test,ax=ax[1]);
ax[1].plot(y_test,y_test, color= "black")
ax[1].set_xlabel("Real Price")
ax[1].set_ylabel("Predicted Price")
ax[1].set_title("Test Set")
plt.tight_layout()

ax[0].ticklabel_format(axis="x",style="sci",scilimits=(0,0))
ax[0].ticklabel_format(axis="y",style="sci",scilimits=(0,0))
ax[1].ticklabel_format(axis="x",style="sci",scilimits=(0,0))
ax[1].ticklabel_format(axis="y",style="sci",scilimits=(0,0))
plt.show()

In [ ]:
#I created and check how far away predictions are from real values (looks gaussian distribution)
residuals = y_train - y_pred_train
sns.distplot(residuals)

### Confidence interval

In [ ]:
def confidence_interval_proportion(confidence_level):
    n = len(df)
    p = df["price"].mean()
    t = stats.t.ppf(confidence_level + (1-confidence_level)/2, df=n-1)
    error = t*df.price.std()/m.sqrt(n)
    confidence_interval = [p - error, p+error]
    return confidence_interval

In [ ]:
#Confidende interval proportion 0.95
CI_095= confidence_interval_proportion(0.95)
CI_095

In [ ]:
#Confidence_interval_proportion 0.80
CI_08= confidence_interval_proportion(0.80)
CI_08

#### Export of dataset

In [ ]:
#Save clean dataframe to a new .csv file to be used in further analysis
df.to_csv("Perth_final_project.csv")